<a href="https://colab.research.google.com/github/deepshikharbhardwaj/Natural_Language_Processing/blob/main/BiDirectional_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
import torch
import torchvision
import torch.nn as nn                        #All N.N. modules, nn.Linear, nn.Conv2d, Batch Norm, Loss Functions
import torch.optim as optim                  #SGD, Adam etc
import torch.nn.functional as F              #All functions that doesnt have any parameters
from torch.utils.data import DataLoader      #Give easier dataset management and creates mini batches
import torchvision.datasets as datasets      #Standard Datasets
import torchvision.transforms as transforms

#Hyperparameters

In [2]:
input_size = 28
sequence_length = 28
num_classes = 10
num_layers = 2
hidden_size = 256
learning_rate = 0.001
batch_size = 64
num_epochs = 2

#Create BiDirectional LSTM

In [3]:
#batch, channel, 28 , 28
#IN RNN -> Consider as 28 sequences with 28 features
class BRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(BRNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True, bidirectional = True) #batch should be the first dimension #lstm - lstm
        self.fc = nn.Linear( hidden_size*2, num_classes ) #hidden state Forward + hidden state Backwawrd CONCATENATED
        
    def forward(self, x):
        h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)

        #Forward Propogation
        out, _ = self.lstm(x, ( h0,c0) )  #hidden state and cell state
        #out = out.reshape(out.shape[0], -1) #dont need all hidden states information
        #out = self.fc(out)
        out = self.fc(out[:,-1,:]) #take all mini batch examples at same time, take LAST HIDDEN STATE, take all features
        return out


#Set Device

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Load Data

In [ ]:
train_dataset = datasets.MNIST(root = '/content', train = True , transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dataset, batch_size = batch_size, shuffle = True)
test_dataset = datasets.MNIST(root = '/content', train = False , transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)

#Initialize Network

In [6]:
model = BRNN(input_size, hidden_size, num_layers, num_classes).to(device)

#Loss and Optimizer

In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

#Train Network

In [8]:
for epoch in range(num_epochs): #In one epoch model has seen all the images once
    for batch_idx , (data, targets) in enumerate(train_loader):
        #Get data to cuda
        data = data.to(device).squeeze(1)  #batch x 1 x 28 x 28 --> RNN need batch x 28 x 28 i.e. sequeeze axis 1
        targets = targets.to(device)

        #forward
        scores = model(data)
        loss = criterion(scores, targets)

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent or adam step
        optimizer.step() #update weights based on the gradients

#Check Accuracy

In [9]:
def check_accuracy(loader, model):
    if loader.dataset.train:
        print('Checking Accuracy on Training Data')
    else:
        print('Checking Accuracy on Test Data')
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device).squeeze(1) #RNN Changes from NN
            y = y.to(device=device)
            #x = x.reshape(x.shape[0], -1) #RNN desired input unlike NN

            scores = model(x)
            values, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0) #0th dimension in [64x10] i.e. 64
        print(f'Got {num_correct}/ {num_samples} with accuracy {float(num_correct)/float(num_samples)*100:.2f} \n')

    model.train()

check_accuracy(train_loader, model)
check_accuracy(test_loader, model)



Checking Accuracy on Training Data
Got 58438/ 60000 with accuracy 97.40 

Checking Accuracy on Test Data
Got 9739/ 10000 with accuracy 97.39 

